In [1]:
import videopipeline as vpl
import os
import numpy as np

def add_suffix(path, suffix, new_ext='.mp4'):
    return path.replace('.mp4', f'-{suffix}{new_ext}')

def draw(images):
    ref = images[0].shape
    assert len(images) == 10, images.shape
    out_image = np.zeros((ref[0] * 2, ref[1] * 5, 3))
    asdf = [images[:5], images[5:]]
    for i, row in enumerate(asdf):
        for j, img in enumerate(row):
            out_image[i * ref[0] : (i+1) * ref[0], j * ref[1] : (j+1) * ref[1]] = img
    
    return out_image

In [2]:
video_path = r"videos/1.mp4"
img_size = (800, 1000)
crop_pos = (200, 800)

video_path = r"videos/2.mp4"
img_size = (1000, 1000)
crop_pos = (0, 800)

video_path = r"videos/3.mp4"
img_size = (1000, 1000)
crop_pos = (0, 800)

video_path = r"videos/4.mp4"
img_size = (1000, 1000)
crop_pos = (0, 800)

video_path = r"videos/5.mp4"
img_size = (1000, 1000)
crop_pos = (0, 800)

In [ ]:
# Absprung detector
pipeline = [
    vpl.nodes.generators.ReadVideoFile(video_path),
    vpl.nodes.functions.Greyscale(),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "gray"), 30),
    vpl.nodes.functions.Crop(crop_pos, img_size),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "crop"), 30),
    vpl.nodes.functions.Smooth(101),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "smooth1"), 30),
    vpl.nodes.functions.AbsDiff(),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "absdiff"), 30),
    vpl.nodes.functions.Threshold(32),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "threshold"), 30),
    vpl.nodes.functions.Dilate(100),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "dilate"), 30),
    vpl.nodes.functions.Erode(100),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "erode"), 30),
    vpl.nodes.functions.Smooth(11),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "smooth2"), 30),
    vpl.nodes.functions.EdgeFilter(1, 100),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "edge"), 30),
    vpl.nodes.functions.FindContours(),
    vpl.nodes.functions.DrawContourCenters(),
    vpl.nodes.functions.DrawMovementPath(),
    vpl.nodes.actions.VideoWriter(add_suffix(video_path, "path"), 30),
    vpl.core.Action(lambda *args: None, aggregate=True, collect=False),
]

# Combine all videos to one
gray = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "gray"))
crop = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "crop"))
smooth1 = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "smooth1"))
absdiff = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "absdiff"))
threshold = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "threshold"))
dilate = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "dilate"))
erode = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "erode"))
smooth2 = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "smooth2"))
edge = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "edge"))
path = vpl.nodes.generators.ReadVideoFile(add_suffix(video_path, "path"))
draw_all = vpl.core.Function(draw)([crop, smooth1, absdiff, threshold, dilate, erode, smooth2, edge, path])
combined = vpl.nodes.actions.VideoWriter(add_suffix(video_path, "combined"), 5, aggregate=True, collect=False)(draw_all)

_ = vpl.core.Pipeline(pipeline)()
_ = vpl.core.Pipeline(combined)()